# Finding all electronic configurations

The recommended way to run this notebook is to click the "Run" button at the top of this page, and select "Run on Binder". This will run the notebook on mybinder.org, a free online service for running Jupyter notebooks. Alternatively, you could install Python and download this notebook on your computer to run it locally. The [conda](https://www.anaconda.com/products/individual#Downloads) distribution of python is recommended for this.

In this notebook we try to find all possible electronic configurations of a system with equidistant energy levels as shown in [this](https://ecee.colorado.edu/~bart/book/book/chapter2/ch2_5.htm) chapter at section [2.5.2](https://ecee.colorado.edu/~bart/book/book/chapter2/ch2_5.htm#2_5_2).

![](https://ecee.colorado.edu/~bart/book/book/chapter2/gif/fig2_5_2.gif)
The example and the image has been taken from chapter 2 of the book [Principles of Semiconductor Devices](https://ecee.colorado.edu/~bart/book/book/append/quick.htm).
All credits goes to the respective owners.

First we import the necessary libraries

In [1]:
import numpy as np
from sympy.utilities.iterables import variations

Now we provide values for the number of electrones `n` and total energy `e`. If the values you entered are invalid (has no possible configurations), a warning pops up and you probably need to recheck your input values. In our case, `n=20` and `e=106`, which are perfectly valid.

In [2]:
n = 20
e = 106

if (n%2==0 and e%1==0) or (n%2==1 and e%1==0.5):
    pass
else:
    print('Invalid values')

The way in which we are going to show the possible electronic configurations is by using a numpy array wherein each element corresponds to the number of electrons in a particular energy level (We can do this because electrons are indistinguishable from each other). We start by filling up electrons in the lowest levels. If the number of electrons is odd, an additional energy level is created to account for the extra electron.

In [3]:
electron_states = np.ones(n//2)*2
if not n%2==0: 
    electron_states = np.append(electron_states,1)

electron_states

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

We also set up another array with energies of the energy levels.

In [4]:
energy = np.arange(0.5,0.5+len(electron_states))

energy

array([0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5])

After the electrons are filled up, we calculate the ground state energy by taking a dot product between the energies and the number of electrons of the energy levels. This is then compared with the given energy. If the given energy turns out to be less than the ground state energy, a warning pops up and you will need to recheck your input values. In the special case wherein the given energy is equal to the ground state energy, the only possible configuration is the ground state configuration which is redundant.

In [5]:
ground_state_energy = np.dot(electron_states,energy)

if e < ground_state_energy:
    print('Invalid values')
elif e == ground_state_energy:
    print('Given energy is ground state energy')
    
ground_state_energy

100.0

As you may have noticed already, there is a maximum energy level `max_en` upto which the electron could go. Also, in many cases, electrons at the lowest levels do not take part in this whole process. The energy level upto which electrons are present which do not take part in this process is `min_en` (It is decremented by one in the case of odd electrons). This extra step is required to make the computational process easier for our computer. If `min_en` is not valid, we set it to zero.

In [6]:
max_en = int(len(electron_states) + (e - ground_state_energy))
min_en = int(len(electron_states) - (e - ground_state_energy))
if n%2==1: min_en -= 1
    
min_en = min_en if min_en>0 else 0
print('Max:',max_en,'\nMin:',min_en)

Max: 16 
Min: 4


Now we extend the `electron_state` array and the `energy` array to a length of `max_en`.

In [7]:
while len(electron_states) < max_en:
    electron_states = np.append(electron_states,0)
    
electron_states

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0.])

In [8]:
energy = np.arange(0.5,0.5+len(electron_states))

energy

array([ 0.5,  1.5,  2.5,  3.5,  4.5,  5.5,  6.5,  7.5,  8.5,  9.5, 10.5,
       11.5, 12.5, 13.5, 14.5, 15.5])

Before we slice the initial few elements of the `energy` array which are not required for the computation, we store those values in another variable `init_energy` which we will use later on. If `min_en` is zero, we set it to a zero matrix.

In [9]:
temp_energy = energy[:min_en]
if min_en == 0: temp_energy = np.zeros(1)
    
temp_energy   

array([0.5, 1.5, 2.5, 3.5])

In [10]:
energy = energy[min_en:]

energy

array([ 4.5,  5.5,  6.5,  7.5,  8.5,  9.5, 10.5, 11.5, 12.5, 13.5, 14.5,
       15.5])

Now we use the `variations` method from the sympy library to calculate all possible configurations of 0, 1 and 2 electrons occupying the energy levels such that, the sum of all electrons should be `n` and the total energy of the electrons should be `e`. We have also accounted for the energy levels we have ignored before in the process. Documentation on the `variations` method can be found [here](https://docs.sympy.org/latest/modules/utilities/iterables.html#variations).

Finally, we sort the results for better readability and print out all possible configurations.

In [11]:
%%time
result = list()
for i in variations([0,1,2],len(energy),True):
    if sum(i) == n-(min_en*2) and np.dot(i,energy) == e - np.dot(temp_energy,np.ones(len(temp_energy))*2):
        result.append(list(np.append(np.ones(min_en)*2,i).astype(int)))

result.sort(reverse=True)
print('There are {} configurations'.format(len(result)))
for i in result:
    print(i)

There are 24 configurations
[2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 0, 0, 0, 1]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 0, 0, 0, 1, 0]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 1, 0, 1, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 0, 0, 0, 1, 0]
[2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 0, 0, 1, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 0, 1, 1, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 0, 1, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 2, 0, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 0, 1, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 1, 0, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 0, 0, 1, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 0, 1, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 0, 2, 0, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 1, 0, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 0, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 0, 1, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 0, 0

Hope that helped. Any sort of feedbacks or comments would be highly appreciated :)

Produced by [Vivek K R](https://www.linkedin.com/in/imvivek98/) (vkr885@gmail.com)

In [ ]:
import jovian
jovian.commit(project='electronic-configuration')

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
